## Щербаченко Елена

In [1]:
import json
import pandas as pd
import numpy as np

import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import string
import re
from pymystem3 import Mystem

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from sklearn.linear_model import LogisticRegression

# import fasttext
from catboost import CatBoostClassifier
from tqdm import tqdm_notebook as tqdm

Поменяйте путь к 3-м файлам, если это необходимо

In [2]:
# загружаем данные:
with open(r'tn.json', 'r', encoding="utf8") as f:
    data_tn = json.loads(f.read())

with open(r'tp.json', 'r', encoding="utf8") as f:
    data_tp = json.loads(f.read())
    
with open(r'test_data.json', 'r', encoding="utf8") as f:
    data = json.loads(f.read())
test_df = pd.DataFrame(data)

# Создаем метки класса
df_tp = pd.DataFrame(data_tp, columns=['sentences'])
df_tp['event'] = np.ones(len(data_tp)).astype(int)

df_tn = pd.DataFrame(data_tn, columns=['sentences'])
df_tn['event'] = np.zeros(len(data_tn)).astype(int)

train_df = pd.concat([df_tn, df_tp], ignore_index = True)

In [3]:
# Делим на тренировочную и тестовую часть
X_train, X_test, y_train, y_test = train_test_split(train_df.sentences, train_df.event, test_size=0.2, random_state = 17)

In [4]:
X_train.shape, X_test.shape, train_df.shape

((1335,), (334,), (1669, 2))

In [5]:
print(train_df.event.value_counts(), '\n', y_train.value_counts(), '\n', y_test.value_counts())

0    1340
1     329
Name: event, dtype: int64 
 0    1074
1     261
Name: event, dtype: int64 
 0    266
1     68
Name: event, dtype: int64


Данных для обучения мало. Классы несбалансированные, поэтому в качестве метрики возьмем F меру, beta > 1 т.к. recall нам важнее (т.е. не пропустить актуальную новость)

## Create preprocessed df

Будем использовать несколько способов препроцессинга данных: удаление специальных символов, стоп слов, и чисел, а так же будем выбирать между стеммингом и лемматизацией; Count векторайзером и TF-IDF векторайзером.

In [6]:
m = Mystem()
def preprocessing(data):
    a = data.copy()
    for i in tqdm(a.index):
        # remove special characters
        a[i] = re.sub(r'\W', ' ', a[i])
        #remove numbers
        a[i] = re.sub(r'\d', '', a[i].lower())
        # Substituting multiple spaces with single space
        a[i] = re.sub(r'\s+', ' ', a[i], flags=re.I)

        a[i] = m.lemmatize(a[i])
        a[i] = ''.join(a[i])
    return a

In [7]:
stopwords_ru = stopwords.words("russian")

In [8]:
X_train_prep = preprocessing(X_train)
# X_train_prep.to_pickle('lem_prep_train_df.pickle')
# X_train_prep = pd.read_pickle('lem_prep_train_df.pickle')

X_test_prep = preprocessing(X_test)
# X_test_prep.to_pickle('lem_prep_test_df.pickle')
# X_test_prep = pd.read_pickle('lem_prep_test_df.pickle')

In [9]:
def tfidf_vect(X_train, X_test, stopwords = None):
    vectorizer = TfidfVectorizer(analyzer='word', max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    X_train_tfidf = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf, columns=vectorizer.get_feature_names())
    X_test_tfidf = pd.DataFrame.sparse.from_spmatrix(X_test_tfidf, columns=vectorizer.get_feature_names())
    return X_train_tfidf, X_test_tfidf

In [10]:
def count_vect(X_train, X_test, stopwords = None, ngram = (1, 1)):
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords, ngram_range=ngram)
    X_train_count = vectorizer.fit_transform(X_train)
    X_test_count = vectorizer.transform(X_test)
    X_train_count = pd.DataFrame.sparse.from_spmatrix(X_train_count, columns=vectorizer.get_feature_names())
    X_test_count = pd.DataFrame.sparse.from_spmatrix(X_test_count, columns=vectorizer.get_feature_names())
    return X_train_count, X_test_count

In [11]:
stemmer = SnowballStemmer("russian", ignore_stopwords=True)
def do_stemming(data):
    data_stem = data.copy()
    for i in tqdm(data_stem.index):
        data_stem[i] = re.sub(r'\W', ' ', data_stem[i])
        data_stem[i] = re.sub(r'\d', '', data_stem[i].lower())
        text = []
        tokens = [token for token in word_tokenize(data_stem[i]) if token not in stopwords_ru and token.strip() not in string.punctuation+'«—»']
        for token in tokens:
            text.append(stemmer.stem(token))
        data_stem[i] = ' '.join(text)
    return data_stem

X_train_stem = do_stemming(X_train)
X_test_stem = do_stemming(X_test)

## CountVect + LogReg
Самая простая модель, бэйзлайн от которого будем отталкиваться

In [12]:
X_train_count, X_test_count = count_vect(X_train, X_test)

model = LogisticRegression()
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.49844236760124605

## CountVec + nltk stopwords + LogReg
Добавляем удаление стоп слов внутри метода CountVectorizer

In [13]:
X_train_count, X_test_count = count_vect(X_train, X_test, stopwords=stopwords_ru)

model = LogisticRegression()
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.45597484276729566

## Stemming + CountVec(with nltk stopwords) + LogReg

In [14]:
X_train_count, X_test_count = count_vect(X_train_stem, X_test_stem, stopwords_ru)

model = LogisticRegression()
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5434782608695652

## Stemming (with nltk stopwords) + TF-IDF Vec + LogReg

In [15]:
X_train_tfidf, X_test_tfidf = tfidf_vect(X_train_stem, X_test_stem, stopwords_ru)

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.3166666666666667

## Lemmatization + TF-IDF Vec + LogReg

In [16]:
X_train_tfidf, X_test_tfidf = tfidf_vect(X_train_prep, X_test_prep, stopwords_ru)

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.33222591362126247

## Lemmatization + Count Vec + LogReg

In [17]:
X_train_count, X_test_count = count_vect(X_train_prep, X_test_prep, stopwords_ru)

model = LogisticRegression()
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5572755417956656

## Lemmatization + Count Vec + CatBoost

In [18]:
X_train_count, X_test_count = count_vect(X_train_prep, X_test_prep, stopwords_ru)

model = CatBoostClassifier(verbose=False)
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

0.5172413793103449

## Stemming + Count Vec + CatBoost

In [19]:
X_train_count, X_test_count = count_vect(X_train_stem, X_test_stem, stopwords_ru)

model = CatBoostClassifier(verbose=False)
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

0.550314465408805

## Lemmatization + Count Vec (n_gram) + LogReg

In [20]:
X_train_count, X_test_count = count_vect(X_train_prep, X_test_prep, stopwords_ru, (1, 2))

model = LogisticRegression()
model.fit(X_train_count, y_train)
y_pred = model.predict(X_test_count)

fbeta_score(y_test, y_pred, 2)

C:\Users\safre\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5555555555555556

В среднем СatBoost и CountVectorizer сработали лучше, поэтому для итоговой модели будем брать их и стемминг. Стемминг выбирается в целях экономии времени, потому что тестовых данных намного больше и лемматизация будет работать долго.

## Word2Vec

### Для обучения эмбедингов будем использовать данные из тестовой выборки тоже.

In [21]:
test_df.head()

,title,text
0,Уралкуз” изготовил рекордное количество осей,"ПАО ""Уральская кузница"" (входит в Группу ""Мече..."
1,Отключение Ирана от интернета выявило сильные ...,Во второй половине ноября Иран охватили протес...
2,Во Владивостоке количество наружной рекламы до...,Смогут ли навести порядок в беспорядочных рекл...
3,"ФУТБОЛ-Месси получил рекордный шестой ""Золотой...",3 дек (Рейтер) - Новый рекорд Лионеля Месси. Н...
4,«ОРУ под напряжением Луны»: фото амурчанина по...,Фотография амурчанина заняла первое место во в...


В отличие от обучающего набора данных, те данные, которые нам надо разметить представлены не в виде отдельных предложений, а в виде целых абзацев, поэтому разбиваем абзацы на предложения. У нас есть повторяющиеся названия для нескольких строк, поэтому создаем новые названия с пометками, чтобы не потерять из какого абзаца взято предложение, когда мы их разобьем.

In [22]:
# Помечаем совпадающие названия новостей
ind = test_df.loc[test_df.duplicated('title', keep = 'first'), 'title'].index
for i in tqdm(test_df.index):
    if i in ind:
        test_df.loc[i, 'new_title'] = test_df.loc[i, 'title'] + str(i)
    else: test_df.loc[i, 'new_title'] = test_df.loc[i, 'title']

In [23]:
# Разбиваем на предложения, для каждого предложения из одного абзаца дцблируем название абзаца
new_test_df = pd.DataFrame()
for i in tqdm(test_df.index):
    test_sent = nltk.tokenize.sent_tokenize(test_df.text[i], 'russian')
    test_new_title = [test_df.new_title[i]]*len(test_sent)
    new_test_dict = {'title': test_new_title, 'sentences': test_sent}
    a = pd.DataFrame(new_test_dict)
    new_test_df = new_test_df.append(a, ignore_index = True)

In [24]:
new_test_df['stem_sent'] = do_stemming(new_test_df.sentences)
new_test_df.to_pickle('stem_new_test_data.pickle')
# new_test_df = pd.read_pickle('stem_new_test_data.pickle')

In [25]:
data = new_test_df.stem_sent.copy()
data = data.rename('sentences')
# Соединяем все данные, которые у нас есть, чтобы посчитать вектора для слов
data = pd.concat([data, X_train_stem, X_test_stem], ignore_index=True)
data

0         па уральск кузниц вход групп мечел итог месяц ...
1         уралкуз октябр год постав абсолютн рекорд исто...
2                       месяц год отгруж тыс единиц продукц
3                                            поб рекорд год
4                      ве год отгруз тыс штук локомотивн ос
                                ...                        
166820    стро сборн железобетон котор выпуска липецк дс...
166821    министерств просвещен рф должн август подготов...
166822    задержк сдач квартир зелен алле месяц привод с...
166823    общ площад переселя аварийн квартир составля м...
166824    помим роскосмос казан заключ контракт поставк ...
Name: sentences, Length: 166825, dtype: object

In [26]:
data_list = []
for i in tqdm(data.index):
    data[i] = data[i].split(' ')
    data_list.append(data[i])

In [27]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(data_list, sg=0, size=100, workers=2)

Wall time: 20.9 s


## Stemming + CounVect + WordEmb+ CatBoost

In [28]:
# делим предложения на слова
def split_text_col(col):
    split_col = col.copy()
    for i in split_col.index:
        split_col[i] = col[i].split(' ')
    return split_col

# для каждого слова находим вектор, суммируем все векторы предложения и нормализуем итоговый
def sentence2vec(s):
    seq = np.array([model.wv[w] for w in s if w in model.wv.vocab.keys()])
    v = seq.sum(axis=0)
    v = v / ((v ** 2).sum() + 1e-100) ** 0.5 if seq.shape[0] != 0 else np.ones(100)*1.0/100**0.5
    # v / ((v ** 2).sum() + 1e-100) ** 0.5 нормализует вектор так, 
    # что он не зависит от кол-ва слов в предложении, геометрическая длина вектора = 1
    # 100 - размер вектора
    return v

# Добавляем вектор табличку
def col2vec(col_train, col_test):
    col_train = split_text_col(col_train)
    col_train_vec = np.array([sentence2vec(s) for s in col_train.values]).tolist()
    col_train_vec = pd.Series(col_train_vec)
    col_train_vec = col_train_vec.rename('vector')
    
    col_test = split_text_col(col_test)
    col_test_vec = np.array([sentence2vec(s) for s in col_test.values]).tolist()
    col_test_vec = pd.Series(col_test_vec)
    col_test_vec = col_test_vec.rename('vector')
    return col_train_vec, col_test_vec

In [49]:
X_train_stem_vec, X_test_stem_vec = col2vec(X_train_stem, X_test_stem)
X_train_stem_count, X_test_stem_count = count_vect(X_train_stem, X_test_stem, stopwords_ru)
# Соединяем countvect и word2vec
X_train_to_model = pd.concat([X_train_stem_count, X_train_stem_vec], axis=1)
X_test_to_model = pd.concat([X_test_stem_count, X_test_stem_vec], axis=1)

In [50]:
vectors=[]
for i in range(1, 101):
    vectors.append('vec_{0}'.format(i))

X_train_to_model[vectors] = pd.DataFrame(X_train_to_model.vector.values.tolist(), index= X_train_to_model.index)
X_train_to_model.drop(['vector'], axis=1, inplace=True)
X_test_to_model[vectors] = pd.DataFrame(X_test_to_model.vector.values.tolist(), index= X_test_to_model.index)
X_test_to_model.drop(['vector'], axis=1, inplace=True)

In [46]:
%%time
model_cat = CatBoostClassifier(verbose=False)
model_cat.fit(X_train_to_model, y_train)
y_pred = model_cat.predict(X_test_to_model)

fbeta_score(y_test, y_pred, 2)

Wall time: 2min 39s


0.5607476635514019

In [53]:
X_train_to_model.shape, X_test_to_model.shape

((1335, 837), (334, 837))

In [55]:
y_pred = model_cat.predict(X_train_to_model)
fbeta_score(y_train, y_pred, 2)

0.9807692307692307

## Results

In [32]:
train_df

,sentences,event
0,Причем часть котельных была запущена раньше ср...,0
1,ООО «БалтИнвестСтрой » взял обязательство рань...,0
2,Школа на Южном шоссе на 825 мест также может б...,0
3,Школу на Южном шоссе планируется сдать на меся...,0
4,"Беглов перечислил школы, которые в 2019 году с...",0
...,...,...
1664,"Бизнесвумен взыскала с «Интеко» 1,5 млн за сры...",1
1665,"Арбитражный суд Москвы взыскал 1,5 миллиона ру...",1
1666,Трутнев раскритиковал срыв сроков сдачи объект...,1
1667,"Глава австрийской OMV Райнер Зеле говорил, что...",1


In [33]:
train_stem = do_stemming(train_df.sentences)
test_stem = new_test_df['stem_sent']
train_stem.shape, test_stem.shape

((1669,), (165156,))

In [34]:
train_stem_count, test_stem_count = count_vect(train_stem, test_stem, stopwords_ru)
train_stem_count.shape, test_stem_count.shape

((1669, 881), (165156, 881))

In [35]:
train_stem_vec, test_stem_vec = col2vec(train_stem, test_stem)
train_stem_vec.shape, test_stem_vec.shape

((1669,), (165156,))

In [36]:
train_to_model = pd.concat([train_stem_count, train_stem_vec], axis=1)
test_to_model = pd.concat([test_stem_count, test_stem_vec], axis=1)
train_to_model.shape, test_to_model.shape

((1669, 882), (165156, 882))

In [37]:
train_to_model[vectors] = pd.DataFrame(train_to_model.vector.values.tolist(), index= train_to_model.index)
train_to_model.drop(['vector'], axis=1, inplace=True)
test_to_model[vectors] = pd.DataFrame(test_to_model.vector.values.tolist(), index= test_to_model.index)
test_to_model.drop(['vector'], axis=1, inplace=True)
train_to_model.shape, test_to_model.shape

((1669, 981), (165156, 981))

In [38]:
%%time
model_cat = CatBoostClassifier(verbose=False)
model_cat.fit(train_to_model, train_df.event)
y_pred = model_cat.predict(test_to_model).astype('int')
y_proba = model_cat.predict_proba(test_to_model)[:, 1]

Wall time: 2min 43s


In [39]:
results = pd.DataFrame({'y_pred': y_pred, 'y_proba': y_proba}, index=test_to_model.index)

In [41]:
result = pd.concat([new_test_df, results], axis=1)
result.head()

,title,sentences,stem_sent,y_pred,y_proba
0,Уралкуз” изготовил рекордное количество осей,"ПАО ""Уральская кузница"" (входит в Группу ""Мече...",па уральск кузниц вход групп мечел итог месяц ...,0,0.020999
1,Уралкуз” изготовил рекордное количество осей,"""Уралкуз"" в октябре 2019 года поставил абсолют...",уралкуз октябр год постав абсолютн рекорд исто...,0,0.064358
2,Уралкуз” изготовил рекордное количество осей,За 10 месяцев 2019 года отгружено более 12 тыс...,месяц год отгруж тыс единиц продукц,0,0.016204
3,Уралкуз” изготовил рекордное количество осей,"""Мы побили рекорд 2013 года.",поб рекорд год,0,0.011726
4,Уралкуз” изготовил рекордное количество осей,"Тогда за весь год мы отгрузили почти 11,2 тыс....",ве год отгруз тыс штук локомотивн ос,0,0.011136


In [42]:
# возвращаемся к исходным данным и каждому абзацу, 
# в котором есть хоть одно предложение с искомым классом ставим класс 1, 
# там расчитываем среднюю вероятность принадлежности классу 1, для ранжирования.
test_df['y_pred'] = 0
test_df['y_proba'] = 0
for tit in tqdm(result.title.unique()):
    a = result[result.title == tit]
    if a.y_pred.any() == True:
        test_df.loc[test_df.new_title == tit, 'y_pred'] = 1
        test_df.loc[test_df.new_title == tit, 'y_proba'] = a.y_proba.mean()

In [43]:
test_event = test_df[test_df.y_pred == 1]

In [44]:
test_event = test_event.sort_values(by='y_proba', ascending=False)[['title', 'text']]

In [45]:
test_event.to_json('result.json')

## Что можно сделать еще
Первоочередное - разобраться с переобучением

* Настроить гипер параметры модели,в том числе регуляризацию, потому что я переобучилась
* Подать CatBoost валидационный набор данных тоже в целях контроля переобучения
* Поварьировать порог классификации
* Использовать предобученные векторы слов